<a href="https://colab.research.google.com/github/JSJeong-me/Machine_Learning/blob/main/actor_critic_cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing libraries
import gym
import numpy as np
from itertools import count
from collections import namedtuple

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
# Importing PyTorch here

In [3]:
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])

In [4]:
env = gym.make('CartPole-v0') # We make the Cartpole environment here

In [5]:
print("There are {} actions".format(env.action_space.n))

There are 2 actions


In [6]:
# You can move either left or right to balance the pole
# Lets implement the Actor critic network
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(4, 128) # 4 because there are 4 parameters as the observation space
        self.actor = nn.Linear(128, 2) # 2 for the number of actions
        self.critic = nn.Linear(128, 1) # Critic is always 1
        self.saved_actions = []
        self.rewards = []
    def forward(self, x):
        x = F.relu(self.fc1(x))
        action_prob = F.softmax(self.actor(x), dim=-1)
        state_values = self.critic(x)
        return action_prob, state_values

In [7]:
def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)
    m = Categorical(probs)
    action = m.sample()
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
    return action.item()
# In this function, we decide whehter we want the block to move left or right,based on what the model decided

In [8]:
def finish_episode():
    # We calculate the losses and perform backprop in this function
    R = 0
    saved_actions = model.saved_actions
    policy_losses = []
    value_losses =[]
    returns = []
    
    for r in model.rewards[::-1]:
        R = r + 0.99 * R # 0.99 is our gamma number
        returns.insert(0, R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)
    
    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()
        
        policy_losses.append(-log_prob * advantage)
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))
    
    optimizer.zero_grad()
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()
    
    loss.backward()
    optimizer.step()
    
    del model.rewards[:]
    del model.saved_actions[:]

In [9]:
model = ActorCritic()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()

In [10]:
def train():
    running_reward = 10
    for i_episode in count(): # We need around this much episodes
        state = env.reset()
        ep_reward = 0
        for t in range(1, 10000):
            action = select_action(state)
            state, reward, done, _ = env.step(action)
            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break
        running_reward = 0.05 * ep_reward + (1-0.05) * running_reward
        finish_episode()
        if i_episode % 10 == 0: # We will print some things out
            print("Episode {}\tLast Reward: {:.2f}\tAverage reward: {:.2f}".format(
                i_episode, ep_reward, running_reward
            ))
        if running_reward > env.spec.reward_threshold:
            print("Solved, running reward is now {} and the last episode runs to {} time steps".format(
                    running_reward, t
            ))
            break
            # This means that we solved cartpole and training is complete
                  

In [11]:
train()

Episode 0	Last Reward: 19.00	Average reward: 10.45
Episode 10	Last Reward: 9.00	Average reward: 10.44
Episode 20	Last Reward: 56.00	Average reward: 15.05
Episode 30	Last Reward: 19.00	Average reward: 17.44
Episode 40	Last Reward: 100.00	Average reward: 26.57
Episode 50	Last Reward: 108.00	Average reward: 57.56
Episode 60	Last Reward: 77.00	Average reward: 74.37
Episode 70	Last Reward: 200.00	Average reward: 117.99
Episode 80	Last Reward: 32.00	Average reward: 120.32
Episode 90	Last Reward: 200.00	Average reward: 141.32
Episode 100	Last Reward: 63.00	Average reward: 158.01
Episode 110	Last Reward: 200.00	Average reward: 163.33
Episode 120	Last Reward: 200.00	Average reward: 178.04
Episode 130	Last Reward: 200.00	Average reward: 186.85
Episode 140	Last Reward: 200.00	Average reward: 192.13
Episode 150	Last Reward: 165.00	Average reward: 189.23
Episode 160	Last Reward: 200.00	Average reward: 175.96
Episode 170	Last Reward: 70.00	Average reward: 179.11
Episode 180	Last Reward: 200.00	Avera

In [12]:
# There. we finished
# Lets see it in action
done = False
cnt = 0

In [15]:

!apt-get install -y xvfb x11-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libxxf86dga1
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 x11-utils xvfb
0 upgraded, 3 newly installed, 0 to remove and 42 not upgraded.
Need to get 993 kB of archives.
After this operation, 2,982 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.10 [784 kB]
Fetched 993 kB in 2s (520 kB/s)
Selecting previously unselected package libxxf86dga1:amd64.
(Read

In [16]:
!pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

     |████████████████████████████████| 538 kB 3.9 MB/s 
     |████████████████████████████████| 448 kB 38.7 MB/s 
  Created wheel for PyOpenGL-accelerate: filename=PyOpenGL_accelerate-3.1.5-cp37-cp37m-linux_x86_64.whl size=1599538 sha256=f6d22bebd0d6c748d1ab3b5ff3580d67818c67b801eb96bb96dd42480ff81b71
  Stored in directory: /root/.cache/pip/wheels/1c/f5/6f/169afb3f2d476c5e807f8515b3c9bc9b819c3962316aa804eb
Successfully built PyOpenGL-accelerate


In [20]:
!pip uninstall pyglet
!pip uninstall gym

Found existing installation: pyglet 1.5.0
Uninstalling pyglet-1.5.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/pyglet-1.5.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pyglet/*
Proceed (y/n)? y
  Successfully uninstalled pyglet-1.5.0
Found existing installation: gym 0.17.3
Uninstalling gym-0.17.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/gym-0.17.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/gym/*
Proceed (y/n)? y
  Successfully uninstalled gym-0.17.3


In [21]:
!conda install -c conda-forge pyglet
!conda install -c conda-forge gym

/bin/bash: conda: command not found
/bin/bash: conda: command not found


In [19]:
observation = env.reset()
while True:
    cnt += 1
    env.render()
    action = select_action(observation)
    observation, reward, done, _ = env.step(action)
    # Lets see how long it lasts until failing
print(f"Game lasted {cnt} moves")

NameError: ignored

In [ ]:
# There We are done